# Imports/setup

In [214]:
import ee
import os
import pandas as pd
import geopandas as gpd
# import json
import geemap
import time
import functools
import os
import sys
# from datetime import datetime

import requests # may not be needed
from sidecar import Sidecar

from modules.gee_initialize import initialize_ee 
initialize_ee()

from modules.file_to_ceo import get_ceo_url, whisp_stats_shapefile_and_ceo_url
from parameters.config_output_naming import (
    geo_id_column, 
    out_file_wide, 
    out_shapefile
)
from modules.agstack_to_gee import (
    start_agstack_session,
    get_agstack_token,
    geometry_to_geo_id,
    feature_to_geo_id,
    geo_id_or_ids_to_feature_collection,
    shapefile_to_ee_feature_collection,
    register_feature_and_set_geo_id,
    register_fc_and_set_geo_id,
    register_feature_and_append_to_csv,
    register_fc_and_append_to_csv,
    add_geo_ids_to_feature_col_from_lookup_df,
    add_geo_ids_to_feature_col_from_lookup_csv
)

from modules.multiband_stats import get_stats,reformat_whisp_fc,get_stats_formatted

from parameters.config_asr_url_info import asset_registry_base, user_registry_base

from parameters.config_asr_credentials import email, password

from modules.utils import (
    collection_properties_to_df,
    remove_geometry_from_feature_collection,
    get_centroid#,
    # get_scale_from_image
)

def whisp_alt_stats_as_df (roi): 
    %store roi
    %run alt_whisp_stats.ipynb   
    return df_out
   

##### Set up asset registry

In [215]:
session = start_agstack_session(email,password,user_registry_base)

token = get_agstack_token(email, password, asset_registry_base)

#### Inputs : with GEOIDS

##### Input: list of Geo ids

In [216]:
example_ids = ['88bec54ad04804f5b1fafbc131266640a129be2840fa6797cda358d7e831b907']

GHA_ids = ['88bec54ad04804f5b1fafbc131266640a129be2840fa6797cda358d7e831b907', 
        '97408ef7bdac487e4a42e4abf20492b786310889fd4b0478603e2d0004c40bfb']

IDN_ids = ['c288d6c94efa9011c0e3452af9f7fa0941661377030e10d29c68764617f9816d', 
       '8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53859993522a6f05727a']

CIV_ids = ['0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f665ea83ac9aad3512918',
            'fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b84f5b9445855eef02f']

all_geo_ids= GHA_ids + IDN_ids + CIV_ids
#all_geo_ids= example_ids

In [217]:
#make into a feature collection
roi_geoid = geo_id_or_ids_to_feature_collection(
    all_geo_ids=all_geo_ids, 
    geo_id_column=geo_id_column, 
    session=session)

#### Inputs : without GEOIDS

##### Input: GEE asset (feature collection) 

In [218]:
example_plot_feature_col_asset = 'projects/ee-andyarnellgee/assets/p0004_commodity_mapper_support/seg_civ_PCBM'
#"projects/fdap-remi/assets/seg_civ_scoopac"
# example_plot_feature_col_asset = "projects/fdap-remi/assets/seg_civ_coopara"
# Define the fraction of features you want to sample (e.g., 0.5 for 50%)
sampleFraction = 0.0010

In [219]:
fc = ee.FeatureCollection(example_plot_feature_col_asset)

randomColumn = 'random'
fc_r = fc.randomColumn(randomColumn,4)

# Filter the feature collection based on the random column to get the sample
fc_sample = fc_r.filter(ee.Filter.lt(randomColumn, sampleFraction));

# fc = ee.FeatureCollection(fc.toList(fc.size()).slice(0,3)) #_sample

fc.size()

fc = fc_sample

fc

In [220]:
register_fc_and_append_to_csv(
    feature_col=fc,
    geo_id_column="Geo_id",
    output_lookup_csv="temp_geo_id_lookup.csv",
    # output_lookup_csv="whisp_output_table.csv",
    join_id_column="system:index",
    token=token,
    session=session,
    asset_registry_base=asset_registry_base,
    override_checks=False,
    remove_temp_csv=False,
    debug=True)


Number without geo ids:7. 
 Processing started...
using existing session
Matched existing field (failed to register). Status code: 400 Using pre-existing geo id:  eb7183ae1c62b147e099de79265607bf1a53cb4ff717d50f8daee403284fd0a2
using existing session79265607bf1a53cb4ff717d50f8daee403284fd0a2 for system:index  added to  temp_geo_id_lookup.csv
Matched existing field (failed to register). Status code: 400 Using pre-existing geo id:  37485ae8d1dd2d047caa0fbc57a72e508e83b50bd3c4c14a990b52b2ff3f2a6c
using existing sessionbc57a72e508e83b50bd3c4c14a990b52b2ff3f2a6c for system:index  added to  temp_geo_id_lookup.csv
Matched existing field (failed to register). Status code: 400 Using pre-existing geo id:  375ba910a586a125ff451e960451990123b31ba0e54fcd73728b3a4a9d23ac65
using existing session960451990123b31ba0e54fcd73728b3a4a9d23ac65 for system:index  added to  temp_geo_id_lookup.csv
Matched existing field (failed to register). Status code: 400 Using pre-existing geo id:  146d51ab4e6a931f392efde0

In [111]:
join_id_column = "system:index"

fc_new = add_geo_ids_to_feature_col_from_lookup_csv(
    fc,
    "temp_geo_id_lookup.csv",
    join_id_column,
    geo_id_column,
    override_checks=False,
    remove_other_properties=True,
    debug=True)


Check passed: feature collection and table same size: 7 rows
dropping rows without values (NaN etc) from input table. Before: 7 After: 7 
 processing...
Finished. 
 New count of Geo_id values in feature collection: 7 (from total of 7 features)


In [112]:
# collection_properties_to_df(fc_new)

In [113]:
#run for feature collection (NB needs speeding up...)
# roi_asset = register_fc_and_set_geo_id(
#     fc,
#     geo_id_column,
#     token,
#     session,
#     asset_registry_base,
#     debug=True
# )

# task.start()

roi_asset = fc_new

print ("done")

done


In [114]:
roi_asset

##### Input: polygon feature collection

In [115]:
geom_1 =ee.Geometry.Polygon(
        [[[102.19867776108482, 3.0835809888660344],
          [102.1988172359536, 3.082670357801012],
          # [102.20001886559227, 3.082078035157836],
          # [102.2013385124276, 3.081745922272887],
          [102.2014355657636, 3.0825815609469625],
          [102.20011591892828, 3.0836528916202233]]],None, False);

geom_2 = ee.Geometry.Polygon(
        [[[102.18777620396249, 3.079459767638651],
          # [102.18831264576546, 3.0785384196951506],
          [102.1895786484205, 3.0792026473624907],
          [102.18899929127328, 3.08005971470826]]],None, False);

feat_1 = ee.Feature(geom_1)
feat_2 = ee.Feature(geom_2)

In [116]:
fc = ee.FeatureCollection([feat_1,feat_2])

In [117]:
#run for feature collection (NB needs speeding up..get geo ids then use map over property setting)
roi_asset = register_fc_and_set_geo_id(
    fc,
    geo_id_column,
    token,
    session,
    asset_registry_base,
    debug=True
)


using existing session
Matched existing field (failed to register). Status code: 400 Using pre-existing geo id:  289e109a6e2d7f4bb25bdb5cd13972f30e77a17351b3717ed7de270835515bdb
using existing session
Matched existing field (failed to register). Status code: 400 Using pre-existing geo id:  7fb6292161b2aef16ed146cf9747aea02d4bd5d6bcc0fcf2e1bedd69928ebdca


##### Input: point feature collection

In [118]:
#get test points from polygon feature collection (NB functionality for registering points in asset registry coming soon)
fc.map(get_centroid)

In [119]:
roi_poly = ee.FeatureCollection(fc.map(get_centroid))

##### Input: shapefile

In [17]:
# example_shapefile = /path_to/my_shapefile
example_shapefile = out_shapefile

In [18]:
roi_shapefile = shapefile_to_ee_feature_collection(example_shapefile)

##### Input: drawn feature

In [19]:
zoom_level =16

Map = geemap.Map(center=[6,-4], zoom=zoom_level)

# Map.add_basemap("Esri.WorldImagery")# optional basemaps from geemap

sc = Sidecar(title='draw polygon')

with sc:display(Map)
print("Draw shape on map before running next cell")

Draw shape on map before running next cell


In [20]:
# #NB User input needed: draw boundary (under 400ha) on the map before running this cell
# try:
#     drawn_feature = (Map.draw_last_feature)
#     if drawnMap.addLayer(drawn_feature)
# except:
#     print("No drawn input. Making feature from map bounds instead") 
#     drawn_feature = ee.Feature(Map.getBounds({'asGEOJSON':True}))
#     Map.addLayer(drawn_feature)
# drawn_feature

In [21]:
#register geo id and add to feature
feature_w_geo_id = register_feature_and_set_geo_id(
    drawn_feature,
    geo_id_column,
    token,
    session,
    asset_registry_base,
    debug=True)

NameError: name 'drawn_feature' is not defined

In [ ]:
roi_drawn = feature_w_geo_id

#### Get plot boundaries

In [221]:
# roi = roi_geoid
roi = fc#roi_asset
#roi = roi_shapefile
#roi = roi_polygon
#roi = roi_points
#roi = roi_drawn

### Whisp it

In [222]:
from parameters.config_runtime import country_dataset_name

from parameters.config_output_naming import geo_id_column, geometry_area_column, plot_id_column

from parameters.config_lookups import lookup_gee_datasets


## get lists from lookup for different streams of processing
from  dataset_properties.make_processing_lists_from_lookup import buffer_stats_list,\
                                                                  presence_only_flag_list,\
                                                                  country_allocation_stats_only_list, \
                                                                  plot_stats_list, \
                                                                  decimal_place_column_list


In [232]:
df = whisp_alt_stats_as_df(roi)

Stored 'roi' (FeatureCollection)
7
...stats processing started
Geo_id column not found, so skipping inclusion
keeping 'system:index' column (e.g. to allow joining Geo ids to df later etc)
...ee to df processing time: 5.984664440155029 seconds


### Display it

In [233]:
df

,system:index,Area ha,Country,EUFO 2020,GLAD Primary,TMF undist,JAXA FNF 2020,GFC TC 2020,GLAD LULC 2020,ESA TC 2020,TMF disturbed,RADD alerts,TMF plant,Oil palm Descals,Oil palm FDaP,Cocoa ETH,WDPA
PLOTID,,,,,,,,,,,,,,,,,
1,000000000000000007b6,0.4,CIV,100,0,31,100,100,100,100,69,-,0,0,0,54,-
2,00000000000000000121,0.7,CIV,100,0,37,100,49,100,100,58,-,0,0,0,3,-
3,00000000000000000df1,3.1,CIV,100,0,81,100,91,100,97,19,-,0,0,0,26,-
4,0000000000000000029d,4.0,CIV,82,0,22,98,86,98,98,73,-,0,0,0,43,-
5,00000000000000000835,4.1,CIV,100,0,52,100,86,100,100,48,-,0,0,0,33,-
6,000000000000000009d2,5.4,CIV,63,0,0,100,76,85,93,47,-,0,0,0,3,-
7,000000000000000014d6,11.2,CIV,94,0,18,95,63,100,98,73,-,0,0,0,1,-


#### Export it to CSV file

In [234]:
#export output csv
df.to_csv(path_or_buf=out_file_wide,header=True)

print ("output csv: ", out_file_wide)


output csv:  /home/sepal-user/whisp/whisp_output_table.csv


#### Add geo ids to CSV

In [226]:
import pandas as pd

def add_empty_column_to_csv(csv_file, column_name):
    df = pd.read_csv(csv_file)

    # Check if the column exists in the DataFrame
    if column_name in df.columns:
        print(f"Column '{column_name}' already exists")
    else:
        # if doesn't exist add it
        df.insert(loc=1, column=column_name, value=None)
        print(f"Column '{column_name}' not found, adding it.")

    # Write the updated DataFrame back to the CSV file
    df.to_csv(csv_file, index=False)


def remove_column_from_csv(csv_file, column_name):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)

    # Check if the column exists in the DataFrame
    if column_name in df.columns:
        # Drop the specified column
        df = df.drop(columns=[column_name])
        print(f"Column '{column_name}' removed successfully.")
    else:
        print(f"Column '{column_name}' not found.")

    # Write the updated DataFrame back to the CSV file
    df.to_csv(csv_file, index=False)


In [227]:
def reorder_column(df, column_name, position):
    # Pop the column to remove it from the DataFrame
    column = df.pop(column_name)
    # Insert the column at the specified position
    df.insert(position, column_name, column)


def add_geo_ids_to_csv_from_lookup_df(
    input_csv,
    geo_id_lookup_df,
    join_id_column="system:index",
    geo_id_column="Geo_id",
    # override_checks=False, # needs implementing
    overwrite=False,
    drop_geo=False,
    debug=False
    ):
    
    input_df = pd.read_csv(input_csv)
    
    # Check if the column exists in the DataFrame
    if geo_id_column in input_df.columns:
        print(f"Column '{geo_id_column}' already exists. No join carried out")
        sys.exit()
    elif join_id_column not in input_df.columns:
        print(f"Column '{join_id_column}' not present to carry out join")
        sys.exit()  
    else:
        #carry out the join     
        input_df_w_geo_ids = input_df.merge(geo_id_lookup_df,on=join_id_column,how="left")
    
    reorder_column(df=input_df_w_geo_ids, column_name=geo_id_column, position=1)
    
    if drop_geo:
        if debug: print ("Dropping geometry column ('drop_geo' set to True)")
        input_df_w_geo_ids = input_df_w_geo_ids.drop(".geo",axis=1)   

    if overwrite:
        out_name = input_csv
    else:    
        out_name = f'copy_{input_csv}'
    
    print ("new csv: ",  input_csv)
    
    input_df_w_geo_ids.to_csv(out_name,index=False)
   
    return None


def add_geo_ids_to_csv_from_lookup_csv(input_csv,
    geo_id_lookup_csv,
    join_id_column="system:index",
    geo_id_column="Geo_id",
    # override_checks=False, # needs implementing
    overwrite=False,
    drop_geo=False,
    debug=False
    ):

    
    geo_id_lookup_df = pd.read_csv(geo_id_lookup_csv)

    add_geo_ids_to_csv_from_lookup_df(
        input_csv,
        geo_id_lookup_df,
        join_id_column=join_id_column,
        geo_id_column=geo_id_column,
        # override_checks=False, # needs implementing
        overwrite=overwrite,
        drop_geo=drop_geo,
        debug=debug
    )

    return None    


In [228]:
add_geo_ids_to_csv_from_lookup_csv(
    input_csv="whisp_output_table.csv",
    geo_id_lookup_csv = "temp_geo_id_lookup.csv",
    # join_id_column="system:index",
    # geo_id_column=geo_id_column,
    overwrite=True,
    # drop_geo=False,
    # debug=False
    )


new csv:  whisp_output_table.csv


In [235]:
register_fc_and_append_to_csv(
    feature_col=fc,
    geo_id_column="Geo_id",
    output_lookup_csv=out_file_wide,
    # output_lookup_csv="whisp_output_table.csv",
    join_id_column="system:index",
    token=token,
    session=session,
    asset_registry_base=asset_registry_base,
    override_checks=True,
    remove_temp_csv=False,
    debug=True)


Check passed: feature collection and table same size: 7 rows


KeyError: 'Geo_id'

In [103]:
# remove_column_from_csv(csv_file=out_file_wide, column_name="system:index")


Column 'system:index' removed successfully.


#### Export it to CEO link

Temporary login details available for demo purposes here: modules/config_ceo.py

In [ ]:
ceo_url = whisp_stats_shapefile_and_ceo_url(roi, df, plot_id_column, out_shapefile)

print('CEO URL:', ceo_url) 